# IESB
## Pós Graduação em Ciência de Dados
## Disciplina - Data Mining e Machine Learning II


**Descrição**

A base de dados "Costa Rican Household Poverty Level" possui dados familiares e de residência de pessoas em condições de pobreza na Costa Rica. Cada linha representa uma pessoa e muitas pessoas podem fazer parte de uma mesma casa. Por isso, somente as predições para o chefe da família são consideradas.

Trata-se de um problema de multi-classificação onde a variável alvo (`Target`) indica a situação de probreza da residência:
1 = pobreza extrema 
2 = pobreza moderada 
3 = situação de vulnerabilidade
4 = situação de não-vulnerabilidade

**Objetivo**

Criar um modelo de ML usando ´RandomForrest´ para prever a situação de cada casa, lembrando que as previsões são consideradas apenas para os chefes de família.

O desempenho do modelo deve ser medido pela métrica de `f1_score`.

In [ ]:
# Importando as bibliotecas
import numpy as np
import pandas as pd

# Idnetificando os arquivos
import os
print(os.listdir("../input"))

In [ ]:
# Carregando os datasets
df = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
# juntando os dataframes
df_all = df.append(test)

In [ ]:
df_all.info()

In [ ]:
# Tamanho da base original de treino
df.shape

In [ ]:
# Tamanho da base original de teste
test.shape

In [ ]:
# Quais as colunas do tipo object?
df_all.select_dtypes('object').head()

In [ ]:
# Vamos transformar 'yes' em 1 e 'no' em 0
mapping = {'yes': 1, 'no': 0}
df_all['edjefa'] = df_all['edjefa'].replace(mapping).astype(np.float64)
df_all['edjefe'] = df_all['edjefe'].replace(mapping).astype(np.float64)

In [ ]:
# Olhando a coluna dependency
df_all['dependency'].value_counts()

In [ ]:
# Vamos transformar 'yes' em 1 e 'no' em 0
df_all['dependency'] = df_all['dependency'].replace(mapping).astype(np.float64)

In [ ]:
# Quais as colunas do tipo object?
df_all.select_dtypes('object').head()

In [ ]:
# Separando as colunas
feats = [col for col in df_all.columns if col not in ['Id', 'idhogar', 'Target']]

In [ ]:
# Verificando os valores negativos
df_all[feats].min().min()

In [ ]:
print(sum(df_all.isna().any()))
print(df_all.isnull().sum())

In [ ]:
# Preenchendos os valores nulos com -1
df_all.fillna(-1, inplace=True)

In [ ]:
# Separando os dataframes
train, test = df_all[df_all['Target'] != -1], df_all[df_all['Target'] == -1]

In [ ]:
# Baseline das previsões
train['Target'].value_counts(normalize=True)

In [ ]:
# Rodando um modelo RandomForrest
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=42, n_estimators=200)

In [ ]:
rf.fit(train[feats], train['Target'])

In [ ]:
# Prevendo os valores para a base de teste
test['Target'] = rf.predict(test[feats]).astype(int)

In [ ]:
# criando o arquivo para submissão
test[['Id', 'Target']].to_csv('submissao.csv', index=False)

In [ ]:
# vamos rodar o modelo apenas com os pais de familia
heads = train[train['parentesco1'] == 1]

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=666, n_estimators=200)

In [ ]:
rf.fit(heads[feats], heads['Target'])

In [ ]:
test['Target'] = rf.predict(test[feats]).astype(int)

In [ ]:
test[['Id', 'Target']].to_csv('heads.csv', index=False)

In [ ]:
# Feature Engineering
# Vamos criar colunas para valores percapita
df_all['house-size-per-capita'] = df_all['hhsize'] / df_all['tamviv']
df_all['phones-per-capita'] = df_all['qmobilephone'] / df_all['tamviv']
df_all['tablets-per-capita'] = df_all['v18q1'] / df_all['tamviv']
df_all['rooms-per-capita'] = df_all['rooms'] / df_all['tamviv']
df_all['rent-per-capita'] = df_all['v2a1'] / df_all['tamviv']

In [ ]:
# Separar treino e test
train, test = df_all[df_all['Target'] != -1], df_all[df_all['Target'] == -1]

In [ ]:
# Separar só pais de familia para treinar o modelo
heads = train[train['parentesco1'] == 1]

In [ ]:
# Separando as colunas
feats = [col for col in df_all.columns if col not in ['Id', 'idhogar', 'Target']]

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=42, n_estimators=200)

In [ ]:
rf.fit(heads[feats], heads['Target'])

In [ ]:
test['Target'] = rf.predict(test[feats]).astype(int)

In [ ]:
test[['Id', 'Target']].to_csv('new_features.csv', index=False)

In [ ]:
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh(figsize = (20,40))

In [ ]:
df_all.reset_index(drop=True, inplace=True)

In [ ]:
from sklearn.feature_selection import VarianceThreshold

train_idx = train.index
test_idx = test.index

cols = feats.copy()
cols.remove('parentesco1')

df2 = VarianceThreshold(threshold=1.6).fit_transform(df_all[cols])

train2 = df2[:train.shape[0]]
test2 = df2[train.shape[0]:]

rf2 = RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=666, n_estimators=200)
rf2.fit(train2, df_all.loc[train_idx]['Target'])

preds = rf2.predict(test2).astype(int)

In [ ]:
pd.Series(rf2.feature_importances_).sort_values().plot.barh(figsize = (10, 15))

In [ ]:
test['Target'] = preds
test[['Id', 'Target']].to_csv('variance.csv', index=False)